## Discovering matrix multiplication algorithms

Here we illustrate how remarkable FedroTensor is at finding fast matrix multiplication algorithms.
Namely, our task is to minimise the number of scalar multiplications.
This problem is equivalent to finding the lowest possible CP rank of a certain 3-dimensional tensor.

Let's start with the 2x2 case. The naive approach results in 8 (=2x2x2) scalar multiplications, but there is a way to do it with only 7 multiplications, known as the Strassen algorithm.
Let's see if our methods can recover it.

In [14]:
from src.tensor_rank import cp_rank
from src.matmul_tensor import build_matmul_tensor

T = build_matmul_tensor(2, 2, 2)
r, factors = cp_rank(T)
r

7

The rank indeed corresponds to the Strassen algorithm cost. Let's see if we can recover the algorithm from the factors.

In [15]:
factors

[array([[ 0.01769191,  0.43929366,  0.53100648,  1.22937847,  0.87743719,
          0.02910171,  1.04992179],
        [-0.04391392, -1.07318842,  0.64202131, -0.26205907, -0.21897871,
          0.03497627, -0.2243904 ],
        [-0.38836249, -0.15485323,  0.30435333, -0.43265363,  0.17889843,
         -0.62831518,  0.6025638 ],
        [ 0.94863385,  0.37700718,  0.36823169,  0.09175423,  0.66659961,
         -0.75986475, -0.12872803]]),
 array([[-0.00475142, -0.17473639, -0.17209083, -0.90783198, -0.36046859,
         -0.00662175, -0.77041184],
        [-0.85691468, -0.060133  ,  0.08424802, -0.31219564, -1.13605913,
         -1.2447182 ,  0.37349623],
        [ 0.00424097, -0.81859572, -0.80879407, -0.37168878, -1.65632527,
         -0.00255727,  0.63648553],
        [ 0.70841483, -0.28168357,  0.39246513, -0.12808374,  0.26700653,
         -0.51008773, -0.30913612]]),
 array([[ 0.17217918,  0.7430965 , -0.66007923, -0.6086962 ,  0.02104419,
          0.16450365, -0.39962625],
      

Not helpful indeed...

Ideally, we would like to have integer factors, or at least the rational ones. We can compute them by specifying `rational=True`:

In [16]:
r, factors = cp_rank(T, rational=True)
factors

[array([[ 0.,  0.,  0.,  1.,  0.,  0., -1.],
        [-1., -1.,  0.,  0.,  0.,  1.,  1.],
        [ 0.,  1., -1., -1.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  0., -1.,  0.,  0.]]),
 array([[ 0.,  0.,  1.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  0., -1.,  0.,  1., -1.],
        [ 1., -1.,  1.,  0., -1.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  1.,  0.]]),
 array([[ 0.,  1., -1.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  1.],
        [ 0.,  0., -1.,  0.,  1.,  0.,  0.],
        [-1., -1.,  0.,  0.,  1.,  1.,  0.]])]

That's much better, as we can now write down the algorithm precisely.

Let's consider the 3x3 case. The best known algorithm (the Laderman's algorithm) has 23 multiplications. Remarkably, our method manages to find rank 23:

In [17]:
T = build_matmul_tensor(3, 3, 3)
r, factors = cp_rank(T)
r

23

Computing rational factors takes slightly longer, but it's definitely worth it:

In [19]:
r, factors = cp_rank(T, rational=True)
factors

[array([[ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.],
        [ 1.,  1.,  0., -1.,  0.,  0.,  0.,  0., -1., -1., -1.,  0.,  0.,
          0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0., -1., -1.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,
          0.,  0.,  0., -1., -1.,  0., -1.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  1.,  0., -1.,  1.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0.,  0.

Finally, it's time for a benchmark on different sizes (n, m, p).

In [21]:
sizes = [(2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 2, 5), (2, 3, 3), (2, 3, 4), (2, 3, 5), (2, 4, 4), (2, 4, 5), (2, 5, 5), (3, 3, 3), (3, 3, 4), (3, 3, 5), (3, 4, 4), (3, 4, 5), (4, 4, 4)]
true_rs = [7, 11, 14, 18, 15, 20, 25, 26, 33, 40, 23, 29, 36, 38, 47, 49]
print('n\tm\tp\tr\ttrue_r')
for (n, m, p), true_r in zip(sizes, true_rs):
    T = build_matmul_tensor(n, m, p)
    r, _ = cp_rank(T)
    print(f'{n}\t{m}\t{p}\t{r}\t{true_r}')

n	m	p	r	true_r
2	2	2	7	7
2	2	3	11	11
2	2	4	14	14
2	2	5	18	18
2	3	3	15	15
2	3	4	20	20
2	3	5	26	25
2	4	4	26	26
2	4	5	34	33
2	5	5	41	40
3	3	3	23	23
3	3	4	31	29
3	3	5	39	36
3	4	4	42	38
3	4	5	52	47
4	4	4	56	49


Notice our method finds optimal ranks when the rank is less than 25. In the larger cases, it's slightly worse (for now). One can increase `num_attempts` from 10 to, say, 100, but then the computation becomes proportionally slower.